<a href="https://colab.research.google.com/github/Jcc329/Jessica_DATA606/blob/main/Cleaned_Data/Steam_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 2: Data Cleaning

This Notebook aims to clean and prep the data for analysis

### Data cleaning

The final dataset contains several columns with many values (subdictionaries).
To handle these, I will identify all columns containing desired data, remove unneccessary columns and use the apply function to convert the multidimentional columns into their own dataframes that can be appened back onto the original dataframe. 

#### Text Cleaning

Any text data will undergo additional cleaning to prepare it for analysis, including converting the text to lowercase, removing symbols and punctuation, and generally tidying the data.

This concludes the goals of this notebook. A cleaned dataset will be save and the next stage of EDA will occur in the next notebook in this series. 


In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import math

In [2]:
#import raw data

GameData = pd.read_csv('RawSteamGameData.csv',  engine='python')

In [3]:
GameData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7309 entries, 0 to 7308
Data columns (total 63 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                7309 non-null   int64  
 1   type                      7309 non-null   object 
 2   name                      7301 non-null   object 
 3   steam_appid               7309 non-null   int64  
 4   required_age              7309 non-null   object 
 5   is_free                   7309 non-null   bool   
 6   detailed_description      6664 non-null   object 
 7   about_the_game            6664 non-null   object 
 8   short_description         7177 non-null   object 
 9   supported_languages       7043 non-null   object 
 10  header_image              7309 non-null   object 
 11  website                   4003 non-null   object 
 12  pc_requirements           7309 non-null   object 
 13  mac_requirements          7309 non-null   object 
 14  linux_re

In [4]:
GameData.isnull().sum()

Unnamed: 0                    0
type                          0
name                          8
steam_appid                   0
required_age                  0
                           ... 
ext_user_account_notice    7243
demos                      6918
metacritic                 7059
drm_notice                 7269
alternate_appid            7308
Length: 63, dtype: int64

In [5]:
#drop fields with a lot of mising data and fields that are unneccessary for the current analysis.
# Dropping 'drm_notice', 'alternate_appid','score_rank', 'ext_user_account_notice', 'demos', 'dlc',  high number of missing data
# Dropping 'legal_notice', 'header_image', 'website', 'pc_requirements', 'packages','mac_requirements', 'linux_requirements', 'screenshots', 'movies', due to irrelevance
#Dropping duplicated developer, publisher
#Only 250 games include metacritic scores. I will keep the field however, as I may decide to use it during exploratory analysis, or if I expand the number of games collected

GameData = GameData[['Unnamed: 0', 'type', 'name', 'steam_appid', 'required_age', 'is_free',
       'detailed_description', 'about_the_game', 'short_description',
       'supported_languages', 'developers', 'publishers',
       'price_overview', 'packages', 'platforms',
       'categories', 'genres', 'release_date','content_descriptors', 'Review Score',
       'Review Score Description', 'Top Reviews by Upvotes', 'appid',
       'positive', 'negative',
       'userscore', 'owners', 'average_forever', 'average_2weeks',
       'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
       'ccu', 'languages', 'genre', 'tags', 'fullgame', 'reviews',
       'achievements', 'dlc', 'controller_support',
       'recommendations','metacritic'
       ]]

## A major issue with this data are the collumns that contain multiple datapoints. There are 3 formats this takes in  the data frame right now:
- a dictionary
- a list stored as a string
- a list of dictionaries stored as strings

I also need to convert the acheivements value into a binary (I don't care what the acheivements are, only whether or not it's a feature in the game). 

In [6]:
#Expand columns containing multiple datapoints
#https://stackoverflow.com/questions/38231591/split-explode-a-column-of-dictionaries-into-separate-columns-with-pandas

# I will start by handling dictionary lists stored as strings 
# create a function to convert the lists of dictionaries stored as strings to dictionaries for processing
# https://www.geeksforgeeks.org/python-convert-string-to-list-of-dictionaries/

def makelist(DictStrList):
    ''' 
    Takes list of dictionaries stored as a string and converts to a list of dictionaries
    '''
    try:    
        if len(DictStrList.split(', '))==2:
            x = eval(DictStrList)
        else:
            x = list(eval(DictStrList))
    except: 
        x = DictStrList
        # print(DictStrList)
    return x

In [7]:
# Handle categories
#desired output 1 column for each category type with a value indicating whether or not the game includes that value
#Start by finding each category name:
GameData2 = GameData
GameData2['categories'] = GameData['categories'].apply(makelist)
GameData2 = GameData.explode('categories')
categories = pd.concat([GameData2.drop(['categories'], axis=1), GameData2['categories'].apply(pd.Series)], axis=1)
uniquecats = list(categories['description'].unique()) #create list of unique categories
uniquecats #remove categories not in english


['Single-player',
 'Downloadable Content',
 'Steam Achievements',
 'Partial Controller Support',
 nan,
 'Multi-player',
 'PvP',
 'Online PvP',
 'Shared/Split Screen PvP',
 'Shared/Split Screen',
 'Cross-Platform Multiplayer',
 'Remote Play Together',
 'Co-op',
 'Online Co-op',
 'Shared/Split Screen Co-op',
 'Full controller support',
 'Steam Trading Cards',
 'Steam Workshop',
 'Steam Cloud',
 'Stats',
 'Steam Leaderboards',
 'Includes level editor',
 'Remote Play on TV',
 'MMO',
 'In-App Purchases',
 'VR Support',
 'Native Steam Controller Support',
 'Captions available',
 'Additional High-Quality Audio',
 'LAN Co-op',
 'Remote Play on Tablet',
 'Game demo',
 'LAN PvP',
 'Remote Play on Phone',
 '单人',
 'Steam 成就',
 '完全支持控制器',
 'Steam 集换式卡牌',
 'Steam 云',
 'Steam Turn Notifications',
 'Commentary available',
 'Valve Anti-Cheat enabled',
 'Mods',
 '部分支持控制器',
 'DLC',
 'SteamVR Collectibles',
 'Includes Source SDK',
 '多人',
 '合作',
 '包含关卡编辑器',
 'Steam 排行榜',
 '玩家对战',
 '线上玩家对战',
 '同屏/分屏玩家对战',
 

In [14]:
#remove categories not in english
remove = ['多人', '合作','部分支持控制器', '包含关卡编辑器', 'Steam 排行榜', '玩家对战', '线上玩家对战', '同屏/分屏玩家对战', '同屏/分屏',
       '统计数据', '远程同乐', '在线合作', 'Steam 创意工坊', '单人', 'Steam 成就', '完全支持控制器', 'Steam 集换式卡牌','Steam 云']
catlist = [cat for cat in uniquecats if cat not in remove]
#remove nan value
catlist = [cat for cat in catlist if pd.isnull(cat) == False]
catlist

['Single-player',
 'Downloadable Content',
 'Steam Achievements',
 'Partial Controller Support',
 'Multi-player',
 'PvP',
 'Online PvP',
 'Shared/Split Screen PvP',
 'Shared/Split Screen',
 'Cross-Platform Multiplayer',
 'Remote Play Together',
 'Co-op',
 'Online Co-op',
 'Shared/Split Screen Co-op',
 'Full controller support',
 'Steam Trading Cards',
 'Steam Workshop',
 'Steam Cloud',
 'Stats',
 'Steam Leaderboards',
 'Includes level editor',
 'Remote Play on TV',
 'MMO',
 'In-App Purchases',
 'VR Support',
 'Native Steam Controller Support',
 'Captions available',
 'Additional High-Quality Audio',
 'LAN Co-op',
 'Remote Play on Tablet',
 'Game demo',
 'LAN PvP',
 'Remote Play on Phone',
 'Steam Turn Notifications',
 'Commentary available',
 'Valve Anti-Cheat enabled',
 'Mods',
 'DLC',
 'SteamVR Collectibles',
 'Includes Source SDK']

In [23]:
#Next iterate through list and create lists for each category to append
def categoryoffered(dfCol, valuelist):
    ''' 
    Takes a data frame column and list of strings and iterates through the column looking for those strings
    returns a dataframe with a column for each value.
    '''
    CategoryDict = {}
    for category in valuelist: #start a list in a dictionary for each category
        CategoryDict[category] = []
    # print(CategoryDict)
    for categories in dfCol:
        print(categories)
        try:
            for category in valuelist:
                if category in categories: #if the game has that category tag, add a 1 to the category column, else add a 0
                    CategoryDict[category].append(1)
                else:
                    CategoryDict[category].append(0)
        except: #handle Nans
            for category in valuelist:
                CategoryDict[category].append(0)
    return pd.DataFrame.from_dict(CategoryDict)


CategoryDF = categoryoffered(GameData['categories'], catlist)
CategoryDF

Streaming output truncated to the last 5000 lines.
[{'id': 2, 'description': 'Single-player'}, {'id': 22, 'description': 'Steam Achievements'}, {'id': 23, 'description': 'Steam Cloud'}, {'id': 44, 'description': 'Remote Play Together'}]
[{'id': 28, 'description': 'Full controller support'}, {'id': 33, 'description': 'Native Steam Controller Support'}]
[{'id': 22, 'description': 'Steam Achievements'}]
[{'id': 2, 'description': 'Single-player'}, {'id': 22, 'description': 'Steam Achievements'}, {'id': 23, 'description': 'Steam Cloud'}]
[{'id': 2, 'description': 'Single-player'}, {'id': 22, 'description': 'Steam Achievements'}, {'id': 23, 'description': 'Steam Cloud'}]
[{'id': 1, 'description': 'Multi-player'}, {'id': 9, 'description': 'Co-op'}, {'id': 27, 'description': 'Cross-Platform Multiplayer'}, {'id': 21, 'description': 'Downloadable Content'}, {'id': 18, 'description': 'Partial Controller Support'}]
[{'id': 2, 'description': 'Single-player'}, {'id': 1, 'description': 'Multi-player'

,Single-player,Downloadable Content,Steam Achievements,Partial Controller Support,Multi-player,PvP,Online PvP,Shared/Split Screen PvP,Shared/Split Screen,Cross-Platform Multiplayer,Remote Play Together,Co-op,Online Co-op,Shared/Split Screen Co-op,Full controller support,Steam Trading Cards,Steam Workshop,Steam Cloud,Stats,Steam Leaderboards,Includes level editor,Remote Play on TV,MMO,In-App Purchases,VR Support,Native Steam Controller Support,Captions available,Additional High-Quality Audio,LAN Co-op,Remote Play on Tablet,Game demo,LAN PvP,Remote Play on Phone,Steam Turn Notifications,Commentary available,Valve Anti-Cheat enabled,Mods,DLC,SteamVR Collectibles,Includes Source SDK
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7304,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7305,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7306,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7307,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
#add category df to GameData
GameData = pd.concat([GameData, CategoryDF])

In [ ]:
#rename and drop extra columns 
GameData = GameData.drop(columns = [0, 'id'])
GameData = GameData.rename({'description':'categories'})

In [ ]:
#handle genres
GameData.reset_index(inplace=True)
GameData.pivot(columns = ['description'], values = ['id'])

In [ ]:
price_data = GameData['price_overview'].apply(pd.Series)
platforms = GameData['platforms'].apply(pd.Series)
categories = pd.DataFrame(GameData['categories'].tolist())
categories